<a href="https://colab.research.google.com/github/pieva/SimPy/blob/main/Una_coda_Bing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Queue Simulation in Python
### Ref: https://www.grotto-networking.com/DiscreteEventPython.html

In [2]:
!pip install simpy

import random
import functools
import simpy
from itertools import accumulate
from tabulate import tabulate

class Packet(object):
 def __init__(self, time, size, id, src="a", dst="z", flow_id=0):
  self.time = time
  self.size = size
  self.id = id
  self.src = src
  self.dst = dst
  self.flow_id = flow_id

 def __repr__(self):
  return "id: {}, src: {}, time: {}, size: {}".format(self.id, self.src, self.time, self.size)

class PacketGenerator(object):
 def __init__(self, env, id,  adist, sdist, initial_delay=0, finish=float("inf"), flow_id=0):
  self.id = id
  self.env = env
  self.adist = adist
  self.sdist = sdist
  self.initial_delay = initial_delay
  self.finish = finish
  self.out = None
  self.packets_sent = 0
  self.action = env.process(self.run())
  self.flow_id = flow_id

 def run(self):
  yield self.env.timeout(self.initial_delay)
  while self.env.now < self.finish:
   yield self.env.timeout(self.adist())
   self.packets_sent += 1
   p = Packet(self.env.now, self.sdist(), self.packets_sent, src=self.id, flow_id=self.flow_id)
   self.out.put(p)

class SwitchPort(object):
 def __init__(self, env, rate, qlimit=None, limit_bytes=True, debug=False):
  self.store = simpy.Store(env)
  self.rate = rate
  self.env = env
  self.out = None
  self.packets_rec = 0
  self.packets_drop = 0
  self.qlimit = qlimit
  self.limit_bytes = limit_bytes
  self.byte_size = 0
  self.debug = debug
  self.busy = 0
  self.action = env.process(self.run())

 def run(self):
  while True:
   msg = (yield self.store.get())
   self.busy = 1
   self.byte_size -= msg.size
   yield self.env.timeout(msg.size*8.0/self.rate)
   self.out.put(msg)
   self.busy = 0
   if self.debug:
    print(msg)

 def put(self, pkt):
  self.packets_rec += 1
  tmp_byte_count = self.byte_size + pkt.size
  if self.qlimit is None:
   self.byte_size = tmp_byte_count
   return self.store.put(pkt)
  if self.limit_bytes and tmp_byte_count >= self.qlimit:
   self.packets_drop += 1
   return
  elif not self.limit_bytes and len(self.store.items) >= self.qlimit-1:
   self.packets_drop += 1
  else:
   self.byte_size = tmp_byte_count
   return self.store.put(pkt)

class PacketSink(object):
 def __init__(self, env, rec_arrivals=False, absolute_arrivals=False, rec_waits=True, debug=False, selector=None):
  self.store = simpy.Store(env)
  self.env = env
  self.rec_waits = rec_waits
  self.rec_arrivals = rec_arrivals
  self.absolute_arrivals = absolute_arrivals
  self.waits = []
  self.arrivals = []
  self.debug = debug
  self.packets_rec = 0
  self.bytes_rec = 0
  self.selector = selector
  self.last_arrival = 0.0
  self.switch1_arrivals = []
  self.switch2_arrivals = []
  self.switch1_waits = []
  self.switch2_waits = []

 def put(self, pkt):
  if not self.selector or self.selector(pkt):
   now = self.env.now
   if self.rec_waits:
    self.waits.append(self.env.now - pkt.time)
   if self.rec_arrivals:
    if self.absolute_arrivals:
     self.arrivals.append(now)
    else:
     self.arrivals.append(now - self.last_arrival)
    self.last_arrival = now

   if pkt.src == "Switch1":
    self.switch1_arrivals.append(pkt.time)
    self.switch1_waits.append(now - pkt.time)
   elif pkt.src == "Switch2":
    self.switch2_arrivals.append(pkt.time)
    self.switch2_waits.append(now - pkt.time)

   self.packets_rec += 1
   self.bytes_rec += pkt.size

   if self.debug:
    print(pkt)

class PortMonitor(object):
 def __init__(self, env, port, dist, count_bytes=False):
  self.port = port
  self.env = env
  self.dist = dist
  self.count_bytes = count_bytes
  self.sizes = []
  self.action = env.process(self.run())

 def run(self):
  while True:
   yield self.env.timeout(self.dist())
   if self.count_bytes:
    total = self.port.byte_size
   else:
    total = len(self.port.store.items) + self.port.busy
   self.sizes.append(total)

adist = functools.partial(random.expovariate, 0.5)
sdist = functools.partial(random.expovariate, 0.01)
port_rate = 1000.0
qlimit=10000
samp_dist = functools.partial(random.expovariate, 1.0)

env = simpy.Environment()

pg = PacketGenerator(env, "Gen1", adist, sdist)
switch_1 = SwitchPort(env, port_rate, qlimit)
switch_2 = SwitchPort(env, port_rate, qlimit)

ps = PacketSink(env, debug=False, rec_arrivals=True, rec_waits=True)

pg.out = switch_1

switch_1.out = switch_2
switch_2.out = ps

pm1 = PortMonitor(env, switch_1, samp_dist)
pm2 = PortMonitor(env, switch_2, samp_dist)

env.run(until=100)

switch1_arrivals = ps.switch1_arrivals
switch2_arrivals = ps.switch2_arrivals
switch1_waits = ps.switch1_waits
switch2_waits = ps.switch2_waits

switch1_departures = [arrival + wait for arrival, wait in zip(switch1_arrivals, switch1_waits)]
switch2_departures = [arrival + wait for arrival, wait in zip(switch2_arrivals, switch2_waits)]

min_len = min(len(switch1_arrivals), len(switch2_arrivals), len(switch1_departures), len(switch2_departures))
switch1_arrivals = switch1_arrivals[:min_len]
switch2_arrivals = switch2_arrivals[:min_len]
switch1_departures = switch1_departures[:min_len]
switch2_departures = switch2_departures[:min_len]

queue_sizes1 = pm1.sizes[:min_len] + [0] * (min_len - len(pm1.sizes))
queue_sizes2 = pm2.sizes[:min_len] + [0] * (min_len - len(pm2.sizes))

waiting_times = ps.waits[:min_len] + [0] * (min_len - len(ps.waits))

all_data = [(queue_sizes1[i], queue_sizes2[i], switch2_arrivals[i], waiting_times[i], switch2_departures[i]) for i in range(min_len)]

print(tabulate(all_data, headers=["Queue Size 1", "Queue Size 2", "Arrival Time", "Waiting Time", "Departure Time"]))


Queue Size 1    Queue Size 2    Arrival Time    Waiting Time    Departure Time
--------------  --------------  --------------  --------------  ----------------
